# Analysis of Workshop 1

WearAQ's first workshop took place at the Tower Hamlets council on Wednesday, March 14th. This notebook runs through some tests with and analysis of the data - both from the participants and the Airbeam device.